# Unit Test in MXNet
---

In MXNet, we use Module package to test optimizers. The first step is to define the data format.

In [12]:
import mxnet as mx
import numpy as np

n_dim = 10
data = np.array([np.ones(n_dim), -np.ones(n_dim)])
label = np.array([1, -1])
batch_size = 2
train_iter = mx.io.NDArrayIter(data, label, batch_size, label_name='linear_output_label')
# 
# fname = mx.test_utils.download('http://archive.ics.uci.edu/ml/machine-learning-databases/letter-recognition/letter-recognition.data')
# data = np.genfromtxt(fname, delimiter=',')[:,1:]
# label = np.array([ord(l.split(',')[0])-ord('A') for l in open(fname, 'r')])
# batch_size = 32
# ntrain = int(data.shape[0]*0.8)
# train_iter = mx.io.NDArrayIter(data[:ntrain, :], label[:ntrain], batch_size, shuffle=True)

Next, we define the neural network

In [13]:
net = mx.sym.Variable('data')
net = mx.sym.FullyConnected(net, name='fc1', num_hidden=1)
net = mx.sym.LinearRegressionOutput(net, name='linear_output')

Create a module

In [22]:
mod = mx.mod.Module(symbol=net,
                    context=mx.cpu(),
                    data_names=['data'],
                    label_names=['linear_output_label'])

In [23]:
# allocate memory given the input data and label shapes
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
# initialize parameters by uniform random numbers
mod.init_params(initializer=mx.init.Constant(value=1))
# use SGD with learning rate 0.1 to train
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))
# use accuracy as the metric
metric = mx.metric.create('mse')

In [24]:
# train 5 epochs, i.e. going over the data iter one pass
for epoch in range(5):
    train_iter.reset()
    metric.reset()
    for batch in train_iter:
        mod.forward(batch, is_train=True)       # compute predictions
        mod.update_metric(metric, batch.label)  # accumulate prediction MSE
        mod.backward()                          # compute gradients
        mod.update()                            # update parameters
        # print mod._optimizer._h_max
    print('Epoch %d, Training %s' % (epoch, metric.get()))

Epoch 0, Training ('mse', 81.0)
Epoch 1, Training ('mse', 1.2789769243681803e-13)
Epoch 2, Training ('mse', 1.4210854715202004e-14)
Epoch 3, Training ('mse', 5.6843418860808015e-14)
Epoch 4, Training ('mse', 1.4210854715202004e-14)


In [25]:
print mod._optimizer.sym

<Symbol linear_output>
